In [ ]:
#  BERT를 활용해 Dense embedding을 구하고 이를 활용해  passage retrieval을 진행

#  필요 패키지 설치

In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")

corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

9606

# 토크나이저 준비 - Huggingface 제공 tokenizer 이용

 hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

tokenizing이 어떻게 되는지 확인

In [ ]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
tokenizer_input = tokenizer(dataset['train'][0]['context'], padding='max_length',truncation=True)
tokenizer.decode(tokenizer_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

# Dense encoder (BERT) 학습 시키기

Dense Encoder 학습을 위한 준비과정

In [ ]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(810)
torch.cuda.manual_seed(810)
np.random.seed(810)
random.seed(810)

In [ ]:
# Training Dataset 준비하기 (question, passage pairs)
# Training Dataset을 준비해 훈련 시 사용
# Question과 Passage는 서로 pair

# Use subset (10 example) of original training dataset
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]
print(len(dataset['train']), len(training_dataset))

60407 5


# Negative sampling

Negative sample은 관련 없는 sample을 뽑아서 embedding space 상에서 멀리 떨어지도록 학습 시킬 때 필요

In [ ]:
# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break

In [ ]:
print('[Positive context]')
print(p_with_neg[0], '\n')
print('[Negative context]')
print(p_with_neg[1], '\n', p_with_neg[2])

[Positive context]
"3.13 만세운동" 은 1919년 3.13일 전주에서 일어난 만세운동이다. 지역 인사들과 함께 신흥학교 학생들이 주도적인 역할을 하며, 만세운동을 이끌었다. 박태련, 김신극 등 전주 지도자들은 군산에서 4일과 5일 독립만세 시위가 감행됐다는 소식에 듣고 준비하고 있었다. 천도교와 박태련 신간회 총무집에서 필요한 태극기를 인쇄하기로 했었다. 서울을 비롯한 다른 지방에서 시위가 계속되자 일본경찰은 신흥학교와 기전학교를 비롯한 전주시내 학교에 강제 방학조치를 취했다. 이에 최종삼 등 신흥학교 학생 5명은 밤을 이용해 신흥학교 지하실에서 태극기 등 인쇄물을 만들었다. 준비를 마친 이들은 13일 장터로 모이기 시작했고, 채소가마니로 위장한 태극기를 장터로 실어 나르고 거사 직전 시장 입구인 완산동과 전주교 건너편에서 군중들에게 은밀히 배부했다. 낮 12시20분께 신흥학교와 기전학교 학생 및 천도교도 등은 태극기를 들고 만세를 불렀다. 남문 밖 시장, 제2보통학교(현 완산초등학교)에서 모여 인쇄물을 뿌리며 시가지로 구보로 행진했다. 시위는 오후 11시까지 서너차례 계속됐다. 또 다음날 오후 3시에도 군중이 모여 만세를 불렀다. 이후 고형진, 남궁현, 김병학, 김점쇠, 이기곤, 김경신 등 신흥학교 학생들은 시위를 주도했다는 혐의로 모두 실형 1년을 언도 받았다. 이외 신흥학교 학생 3명은 일제의 고문에 옥사한 것으로 알려졌다. 또 시위를 지도한 김인전 목사는 이후 중국 상해로 거처를 옮겨 임시정부에서 활동했다. 현재 신흥학교 교문 옆에 만세운동 기념비가 세워져 있다. 

[Negative context]
크누트는 대중에 공개된 지 1년이 후 130kg이 되었다. 크누트와 관객 사이에는 박격포 폭발에도 끄떡 없는 두께 6인치짜리 유리판이 설치되었다. 크누트가 많이 성장하게 되자 2008년 3월 말 크누트의 사육사 중 한 명이었던 마르쿠스 뢰브케는 "크누트의 자립 생활, 독립을 위해 가급적 빨리 크누트를 야생으로 돌려보내야 한다"고 주장하였

# 전처리

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')

In [ ]:
max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

print(p_seqs['input_ids'].size())
# (num_example, pos + neg, max_len)

torch.Size([20, 4, 512])


In [ ]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

# BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러옴

In [ ]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()

  def forward(self, input_ids,attention_mask=None, token_type_ids=None):
      outputs = self.bert(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
      pooled_output = outputs[1]

      return pooled_output

In [ ]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

# Train function 정의 후, 두개의 encoder fine-tuning

In [ ]:
def train(args, dataset, p_model, q_model):

# Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

# Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

# Start training
  global_step = 0

  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()

  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()

      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }

      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}

      p_outputs = p_model(**p_inputs) #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs) #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze() #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1

      torch.cuda.empty_cache()

  return p_model, q_model

In [ ]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01
)

p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  20%|██        | 1/5 [00:01<00:07,  1.97s/it]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  40%|████      | 2/5 [00:03<00:05,  1.97s/it]

...


Iteration:  60%|██████    | 3/5 [00:06<00:04,  2.00s/it]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)



Iteration:  80%|████████  | 4/5 [00:07<00:01,  2.00s/it]

tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)



Epoch: 100%|██████████| 2/2 [00:19<00:00,  9.94s/it]


# Dense Embedding을 활용하여 passage retrieval

우선 sample로 사용한 데이터를 sampling

In [ ]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

print(query, ground_truth)

1999년의 헌법개정에서 강조된 내용은 무엇인가? 중국정부는 중국 본토 전역에 있어서 실효적 지배를 확립하고, 625전쟁에 참전한 후, 1954년에 소비에트연방의 공산권의 선례를 참조하여 최초의 헌법(54년헌법)을 제정하는 등, 소비에트연방법(대륙법계에 속한다)을 계수받은 법제도의 정비를 진행하였다. 그러나, 1957년 6월의 반우파 투쟁을 시작으로 1977년 8월의 문화대혁명의 종결에 이르기까지의 시기에는 “프롤레타리아독재”의 이념에서 도출되는 “중국공산당의 국가에 대한 우위”가 강조되어, 법질서보다도 중국공산당의 정책이 우선되었다. 1978년 3월에는 세 번째의 헌법(78년 헌법)이 제정되었고, 그 후, 인치(人治) 대신에 법치의 필요성이 널리 설명되어, 중국 정부는 형법(1979년), 형사소송법(같은해)을 시작으로 하여 법제도의 정비를 다시 진전시키기 시작하였다. 1982년 12월에는 네 번째의 헌법(82년 헌법)이 제정되었고, 1999년의 헌법개정에서는 사회주의적 법치국가의 건설이 강조되었다. 특히 1992년 10월, 중국공산당 제14기 제3차 회의에서 사회주의시장경제체제의 수립을 경제체제개혁의 목표로 정한 후 중국은 1993년부터 시장경제입법을 본격화하여 1990년대말까지 수년간에 걸쳐 시장경제법률체계의 기본틀을 수립하였다. 이 기간에 제정, 공포된 주요 법률들은 아래와 같다.


In [ ]:
if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

In [ ]:
# passage encoder, question encoder을 이용해 dense embedding 생성

def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [ ]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu') #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze() # (num_passage, emb_dim)
print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


<ipython-input-38-a46e1c15d28c>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  p_embs = torch.Tensor(p_embs).squeeze() # (num_passage, emb_dim)


In [ ]:
# 생성된 embedding에 dot product를 수행하여 Document들의 similarity ranking을 구gka
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

torch.Size([1, 11])


In [ ]:
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

tensor([[67.1352, 51.4858, 64.1141, 48.2257, 55.9088, 62.0516, 67.0191, 56.4848,
         52.5799, 56.0493, 62.9403]])
tensor([ 0,  6,  2, 10,  5,  7,  9,  4,  8,  1,  3])


In [ ]:
# Ranking 중 점수가 높은 5개의 passage를 retrieve 하고 ground truth와 비교
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]], '\n')

[Search query]
 1999년의 헌법개정에서 강조된 내용은 무엇인가? 

[Ground truth passage]
중국정부는 중국 본토 전역에 있어서 실효적 지배를 확립하고, 625전쟁에 참전한 후, 1954년에 소비에트연방의 공산권의 선례를 참조하여 최초의 헌법(54년헌법)을 제정하는 등, 소비에트연방법(대륙법계에 속한다)을 계수받은 법제도의 정비를 진행하였다. 그러나, 1957년 6월의 반우파 투쟁을 시작으로 1977년 8월의 문화대혁명의 종결에 이르기까지의 시기에는 “프롤레타리아독재”의 이념에서 도출되는 “중국공산당의 국가에 대한 우위”가 강조되어, 법질서보다도 중국공산당의 정책이 우선되었다. 1978년 3월에는 세 번째의 헌법(78년 헌법)이 제정되었고, 그 후, 인치(人治) 대신에 법치의 필요성이 널리 설명되어, 중국 정부는 형법(1979년), 형사소송법(같은해)을 시작으로 하여 법제도의 정비를 다시 진전시키기 시작하였다. 1982년 12월에는 네 번째의 헌법(82년 헌법)이 제정되었고, 1999년의 헌법개정에서는 사회주의적 법치국가의 건설이 강조되었다. 특히 1992년 10월, 중국공산당 제14기 제3차 회의에서 사회주의시장경제체제의 수립을 경제체제개혁의 목표로 정한 후 중국은 1993년부터 시장경제입법을 본격화하여 1990년대말까지 수년간에 걸쳐 시장경제법률체계의 기본틀을 수립하였다. 이 기간에 제정, 공포된 주요 법률들은 아래와 같다. 

Top-1 passage with score 67.1352
노무현 정부의 ‘금융허브론’을 계승하여 이명박 정부는 ‘금융중심지론’을 내세웠다. 리먼 브러더스 서울 지점장이었던 민유성이 산업은행장에 임명된 것도 그와 무관하지 않다. 민유성 행장은 산업은행을 민영화하고 세계적인 투자은행(IB)으로 만들 수 있는 능력을 갖춘 것으로 평가받아 2008년 6월 임명됐다. 한편, 리먼 브러더스는 경영 악화 상태에 있었다. 리먼의 위험 자산의 총 규모가 총 500억 달러(50조 원)에 이른다는 분석도 있었고, 200